In [91]:
import numpy as np, numpy as np_test
import pandas as pd,pandas as pd_test
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from tqdm import tqdm
import datetime

una linea


In [92]:
clients = pd.read_csv("./datos_de_clientes.csv", delimiter="\t")
clients.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


una linea


In [93]:
current_year = datetime.datetime.now().year
clients['Age'] = current_year - clients['Year_Birth']

una linea

In [94]:
clients_education_encoded = pd.get_dummies(clients["Education"], prefix="Education", dtype=int)
clients = pd.concat([clients, clients_education_encoded], axis=1)

In [95]:
print(clients['Marital_Status'].value_counts())

Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64


In [96]:
marital_mode = str(clients['Marital_Status'].mode()[0])
print(f"mode: {marital_mode}")
clients['Marital_Status'] = clients['Marital_Status'].replace({'Alone': 'Single', 'Absurd': marital_mode, 'YOLO': marital_mode})
print(clients['Marital_Status'].value_counts())

mode: Married
Marital_Status
Married     868
Together    580
Single      483
Divorced    232
Widow        77
Name: count, dtype: int64


In [97]:
clients_marital_encoded = pd.get_dummies(clients["Marital_Status"], prefix="Marital_Status", dtype=int)
clients = pd.concat([clients, clients_marital_encoded], axis=1)

In [98]:
clients['Days_Customer'] = (pd.to_datetime(datetime.datetime.now().strftime('%d-%m-%Y'), dayfirst=True) - pd.to_datetime(clients['Dt_Customer'], format='%d-%m-%Y')).dt.days

In [102]:
fill = KNNImputer(n_neighbors=1).fit_transform(clients[['Income']])
clients['Income_nuevo'] = pd.DataFrame(fill, columns=['Income'])


KNNImputer(n_neighbors=1)


In [100]:
clients = clients.drop(["ID", "Year_Birth", "Education", "Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue"], axis=1) 
clients.head()

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Days_Customer,Income_nuevo
0,58138.0,0,0,58,635,88,546,172,88,88,...,1,0,0,0,0,1,0,0,4219,58138.0
1,46344.0,1,1,38,11,1,6,2,1,6,...,1,0,0,0,0,1,0,0,3669,46344.0
2,71613.0,0,0,26,426,49,127,111,21,42,...,1,0,0,0,0,0,1,0,3868,71613.0
3,26646.0,1,0,26,11,4,20,10,3,5,...,1,0,0,0,0,0,1,0,3695,26646.0
4,58293.0,1,0,94,173,43,118,46,27,15,...,0,0,1,0,1,0,0,0,3717,58293.0


In [101]:
clients.to_csv("nuevo.csv", index=False)